In [1]:
import os
import re
import importlib
import torch
import json
from langchain.chat_models import ChatOpenAI
from langchain_mistralai import ChatMistralAI
from langchain.schema import HumanMessage
from Utils.Prompt import zeroshot_react_agent_prompt
from typing import List, Dict, Any
from pandas import DataFrame
from transformers import pipeline


OPENAI_API_KEY = os.getenv('OPEN_AI_API')
MISTRAL_API_KEY = os.getenv('MISTRAL_API_KEY')
actionMapping = {"AccommodationSearch":"accommodations", "RestaurantSearch":"restaurants", "AttractionSearch":"attraction","BusinessClusterSearch":"nearby","Planner":"planner"}

class ReactAgent:
    def __init__(self,
                 working_model,
                 react_llm_name,
                 planner_llm_name,
                 #add clustering agent model name here as well
                 mode: str = 'zero_shot',
                 tools: List[str] = None,
                 max_retries: int = 3,
                 ) -> None: 
        self.react_name = react_llm_name
        self.working_model = working_model
        self.answer = ''
        self.json_log = []
        self.mode = mode
        self.planner_name = planner_llm_name
        self.notebook = []
        self.max_retries = max_retries
        self.last_actions = []
        
        self.current_observation = ''
        self.current_data = None

        self.tools = self.load_tools(tools, planner_model_name=planner_llm_name)
        #print(self.tools)
        self.retry_record = {key: 0 for key in self.tools}
        #print(self.retry_record)
        self.retry_record['invalidAction'] = 0
        #print(self.retry_record)

        if self.mode == 'zero_shot':
            self.agent_prompt = zeroshot_react_agent_prompt

        if 'gpt-4o' in react_llm_name:
            stop_list = ['\n']
            self.max_token_length = 15000
            self.llm = ChatOpenAI(temperature=0,
                     max_tokens=256,
                     model_name=react_llm_name,
                     openai_api_key=OPENAI_API_KEY,
                     model_kwargs={"stop": stop_list}
                     )
        if 'mistral' in react_llm_name:
            self.llm = ChatMistralAI(
                    model="mistral-large-2411",
                    max_tokens=128,
                    temperature=0,
                    mistral_api_key = MISTRAL_API_KEY,
                    model_kwargs={"stop": ['\n']}
                )
            
        if 'llama' in react_llm_name:
            self.llm = pipeline(
                    "text-generation", model="meta-llama/Llama-3.1-8B", model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
                )


    def run(self,query,reset=True) -> None:
        self.query = query

        if reset:
            self.__reset_agent()

        while not self.is_finished():
            self.step()
        
        return self.answer, self.scratchpad, self.json_log

    
    def step(self) -> None:
        #record the log
        self.json_log.append({"step": self.step_n, "thought":"",
                              "action": "", "observation": "", "state":""})
        #thought
        self.scratchpad += f'\nThought {self.step_n}:'
        self.scratchpad += ' ' + self.prompt_agent() #
        print(self.scratchpad.split('\n')[-1])
        self.json_log[-1]['thought'] = self.scratchpad.split('\n')[-1].replace(f'\nThought {self.step_n}:',"")
        
        
        #act
        self.scratchpad += f'\nAction {self.step_n}:'
       
        #get the action prompted
        action = self.prompt_agent()
        ##

        if action == None or action == '' or action == '\n':
            self.scratchpad += " Your action is filtered due to content. Please make sure your action does not start with ['\\n', 'Thought', 'Action', 'Observation'] and assume all the actions are permitted in this environment and take the action again."
        else:
            self.scratchpad += ' ' + action

        if len(self.last_actions) > 0 and self.last_actions[-1] != action:
            self.last_actions.clear()

        # refresh last_action list
        self.last_actions.append(action)

        self.json_log[-1]['action'] = self.scratchpad.split('\n')[-1].replace(f'\nAction {self.step_n}:',"")

        if len(self.last_actions) == 3:
            print("The same action has been repeated 3 times consecutively. So we stop here.")
            # self.log_file.write("The same action has been repeated 3 times consecutively. So we stop here.")
            self.json_log[-1]['state'] = 'same action 3 times repeated'
            self.finished = True
            return


        print(self.scratchpad.split('\n')[-1])
        
        
        #observation
        self.scratchpad += f'\nObservation {self.step_n}: '
        action_type, action_arg = parse_action(action)
        #print(action_type, action_arg)
        if action_type != "Planner":
            if action_type in actionMapping:
                pending_action = actionMapping[action_type]
            elif action_type not in actionMapping:
                pending_action = 'invalidAction'

            if pending_action in self.retry_record:
                if self.retry_record[pending_action] + 1 > self.max_retries:
                    action_type = 'Planner'
                    print(f"{pending_action} early stop due to {self.max_retries} max retries.")
                    self.json_log[-1]['state'] = f"{pending_action} early stop due to {self.max_retries} max retries."
                    self.finished = True
                    return # so if the max tries is reached, we stop the loop
            elif pending_action not in self.retry_record:
                if self.retry_record['invalidAction'] + 1 > self.max_retries:
                    action_type = 'Planner'
                    print(f"invalidAction Early stop due to {self.max_retries} max retries.")
                    # self.log_file.write(f"invalidAction early stop due to {self.max_retries} max retries.")
                    self.json_log[-1]['state'] = f"invalidAction early stop due to {self.max_retries} max retries."
                    self.finished = True
                    return

        if action_type == 'AccommodationSearch':
            try:
                if validate_accommodation_parameters_format(action_arg):
                    self.scratchpad = self.scratchpad.replace(to_string(self.current_data).strip(),'Masked due to limited length. Make sure the data has been written in Notebook.')
                    self.current_data = self.tools['accommodations'].run(action_arg.split(',')[0],[p.strip() for p in action_arg.split('[')[1].strip('[]').split(',')])
                    self.current_observation = str(to_string(self.current_data))
                    self.scratchpad += 'AccommodationSearch Succeeded' #self.current_observation
                    self.notebook.append({'Description': 'Accommodation Choice', 'Content': self.current_data})
                    self.__reset_record()
                    self.json_log[-1]['state'] = 'Successful'
                    
            except ValueError as e:
                print(e)
                self.retry_record['accommodations'] += 1
                self.current_observation = str(e)
                self.scratchpad += str(e)
                self.json_log[-1]['state'] = f'Illegal args. Parameter Error'
            except Exception as e:
                print(e)
                self.retry_record['accommodations'] += 1
                self.current_observation = f'Illegal Accommodation Search. Please try again.'
                self.scratchpad += f'Illegal Accommodation Search. Please try again.'
                self.json_log[-1]['state'] = f'Illegal args. Other Error'

        elif action_type == 'AttractionSearch':
            try:
                if validate_attraction_parameters_format(action_arg):
                    self.scratchpad = self.scratchpad.replace(to_string(self.current_data).strip(),'Masked due to limited length. Make sure the data has been written in Notebook.')
                    self.current_data = self.tools['attractions'].run(action_arg.split(',')[0],[action_arg.split(',')[1].strip()[1:][:-1]])
                    self.current_observation = str(to_string(self.current_data))
                    self.scratchpad += 'AttractionSearch Succeeded' #self.current_observation 
                    self.notebook.append({'Description': 'Attraction Choice', 'Content': self.current_data})
                    self.__reset_record()
                    self.json_log[-1]['state'] = f'Successful'
            except ValueError as e:
                print(e)
                self.retry_record['attractions'] += 1
                self.current_observation = str(e)
                self.scratchpad += str(e)
                self.json_log[-1]['state'] = f'Illegal args. Parameter Error'
            except Exception as e:
                print(e)
                self.retry_record['attractions'] += 1
                self.current_observation = f'Illegal Attraction Search. Please try again.'
                self.scratchpad += f'Illegal Attraction Search. Please try again.'
                self.json_log[-1]['state'] = f'Illegal args. Other Error'

        elif action_type == 'RestaurantSearch': #action_arg = 'Cheap Budget, Indian, [Good Flavor, Good Value]'
            try:
                if validate_restaurant_parameters_format(action_arg):
                    self.scratchpad = self.scratchpad.replace(to_string(self.current_data).strip(),'Masked due to limited length. Make sure the data has been written in Notebook.')
                    self.current_data = self.tools['restaurants'].run(action_arg.split('[')[0].split(',')[0].strip(),action_arg.split('[')[0].split(',')[1].strip(),[a.strip() for a in action_arg.split('[')[1].strip()[:-1].split(',')])
                    self.current_observation = str(to_string(self.current_data))
                    self.scratchpad += 'AttractionSearch Succeeded' #self.current_observation
                    self.notebook.append({'Description': 'Restaurant Choice', 'Content': self.current_data})
                    self.__reset_record()
                    self.json_log[-1]['state'] = f'Successful'
            except ValueError as e:
                print(e)
                self.retry_record['restaurants'] += 1
                self.current_observation = str(e)
                self.scratchpad += str(e)
                self.json_log[-1]['state'] = f'Illegal args. Parameter Error'
            except Exception as e:
                print(e)
                self.retry_record['restaurants'] += 1
                self.current_observation = f'Illegal Restaurant Search. Please try again.'
                self.scratchpad += f'Illegal Restaurant Search. Please try again.'
                self.json_log[-1]['state'] = f'Illegal args. Other Error'

        elif action_type == 'BusinessClusterSearch': #action_arg = 'Cheap Budget, Indian, [Good Flavor, Good Value]'
            try:
                self.scratchpad = self.scratchpad.replace(to_string(self.current_data).strip(),'Masked due to limited length. Make sure the data has been written in Notebook.')
                self.current_data = self.tools['nearby'].run(self.notebook)
                self.current_observation = str(to_string(self.current_data))
                self.scratchpad += 'BusinessClusterSearch Succeeded' #self.current_observation
                self.notebook.append({'Description': 'Business Cluster Results', 'Content': self.current_data})
                self.__reset_record()
                self.json_log[-1]['state'] = f'Successful'
            
            except Exception as e:
                print(e)
                self.retry_record['nearby'] += 1
                self.current_observation = f'Illegal business cluster Search. Please try again.'
                self.scratchpad += f'Illegal business cluster Search. Please try again.'
                self.json_log[-1]['state'] = f'Illegal args. Other Error'

        #elif action_type == 'NotebookWrite':
        #    self.scratchpad = self.scratchpad.replace(to_string(self.current_data).strip(),'Masked due to limited length. Make sure the data has been written in Notebook.')
        #    self.current_observation = str(self.tools['notebook'].write(self.current_data, action_arg))
        #    self.scratchpad  +=  self.current_observation
        #    self.json_log[-1]['state'] = f'Successful'

        elif action_type == 'Planner':
            self.current_observation = str(self.tools['planner'].run(self.notebook,action_arg))
            self.scratchpad  +=  self.current_observation
            self.answer = self.current_observation
            self.json_log[-1]['state'] = f'Successful'
        else:
            self.retry_record['invalidAction'] += 1
            self.current_observation = 'Invalid Action. Valid Actions are AccommodationSearch[Budget,Preference] / AttractionSearch[Budget, Preference] / RestaurantSearch[Budget, Cuisine, Preference]/ Planner[Query].'
            self.scratchpad += self.current_observation
            self.json_log[-1]['state'] = f'invalidAction'
        
        #print(f'Observation {self.step_n}: ' + self.current_observation+'\n')
        # rite(f'Observation {self.step_n}: ' + self.current_observation+'\n')
        self.json_log[-1]['observation'] = self.current_observation
        self.step_n += 1

        if action_type and action_type == 'Planner':
            self.finished = True
            self.answer = self.current_observation

            #print(self.scratchpad)
            #print(self.json_log)
            #print(self.notebook) 
        return
    
    
    
    
    def is_finished(self) -> bool:
        return self.finished
    
    #def is_halted(self) -> bool:
    #    return ((self.step_n > self.max_steps) or (
    #                len(self.enc.encode(self._build_agent_prompt())) > self.max_token_length)) and not self.finished
    
    def __reset_agent(self) -> None:
        self.step_n = 1
        self.finished = False
        self.answer = ''
        self.scratchpad: str = ''
        self.__reset_record()
        self.json_log = []
        self.current_observation = ''
        self.current_data = None
        self.last_actions = []
        self.notebook = []

    def prompt_agent(self) -> str:
        #print("we prompt the agents")
        while True:
            if self.react_name == 'gpt-4o-2024-11-20':
                request = format_step(self.llm.invoke(self._build_agent_prompt()).content)
            elif self.react_name == 'mistral-large-2411':
                request = format_step(self.llm.invoke(self._build_agent_prompt()).content.split('\n')[0])
            else:
                request = format_step(self.llm(self._build_agent_prompt(), max_new_tokens = 256, return_full_text=False, do_sample=False)[0]['generated_text'].split('\n')[0])
            #print("here is the raw request: === ", request)
            return request  
        
    def __reset_record(self) -> None:
        self.retry_record = {key: 0 for key in self.retry_record}
        self.retry_record['invalidAction'] = 0

    def _build_agent_prompt(self) -> str:
        if self.mode == "zero_shot":
            return self.agent_prompt.format(
                query=self.query,
                scratchpad=self.scratchpad)
        
    def load_tools(self, tools: List[str], planner_model_name=None) -> Dict[str, Any]:
        tools_map = {}
        for tool_name in tools:
            module = importlib.import_module(f"tools.{tool_name}.apis") #
            
            if tool_name == 'planner' and planner_model_name is not None:
                tools_map[tool_name] = getattr(module, tool_name[0].upper()+tool_name[1:])(model_name=planner_model_name)
            elif tool_name == 'nearby':
                tools_map[tool_name] = getattr(module, tool_name[0].upper()+tool_name[1:])()
            else:
                tools_map[tool_name] = getattr(module, tool_name[0].upper()+tool_name[1:])(working_model = self.working_model)
        #print(tools_map)
        return tools_map
        

def format_step(step: str) -> str:
    return step.strip('\n').strip().replace('\n', '')

def parse_action(string):
    if ('BusinessClusterSearch' not in string):
        pattern = r'^(\w+)\[(.+)\]$'
        match = re.match(pattern, string)
        action_type = match.group(1)
        action_arg = match.group(2)
    else:
        action_type = 'BusinessClusterSearch'
        action_arg = ''
    return action_type,action_arg

#def parse action arg

def to_string(data) -> str:
    if data is not None:
        if type(data) == DataFrame:
            return data.to_string(index=False)
        else:
            return str(data)
    else:
        return str(None)
    
def validate_accommodation_parameters_format(action_arg):
    pattern = r"(.*\s*.*)\s*,\s*\[(.*)\]"
    match = re.match(pattern, action_arg)
    if not match:
        raise ValueError("Parameter format not match. Please try again. Valid Format: Budget, preference list.")
    budget = match.group(1).lower()
    preference_list = match.group(2)

    budget_accepted = ['cheap budget', 'moderate budget','expensive budget']
    budgetInRange = False
    if budget in budget_accepted:
        budgetInRange = True
    if not budgetInRange:
        raise ValueError("Wrong budget Input, valid ones include: cheap budget, moderate budget, and expensive budget. Please try again.")

    #preference
    preference = preference_list.split(',')
    preference_core = [p.lower().strip().split(' ')[-1].strip() for p in preference]
    preferenceInRange = True
    preferenceAccepted = ['location','service','safety','quality']
    for p in preference_core:
        if p not in preferenceAccepted:
            preferenceInRange = False

    if not preferenceInRange:
        raise ValueError("Wrong preference Input. Please try again.")
    return True
    
def validate_attraction_parameters_format(action_arg):
    pattern = r"(.*\s*.*)\s*,\s*\[(.*)\]"
    match = re.match(pattern, action_arg)
    if not match:
        raise ValueError("Parameter format not match. Please try again. Valid Format: Budget, Preference list.")
    budget = match.group(1).lower()
    preference_list = match.group(2)

    budget_accepted = ['cheap budget', 'moderate budget','expensive budget']
    budgetInRange = False
    if budget in budget_accepted:
        budgetInRange = True
    if not budgetInRange:
        raise ValueError("Wrong budget Input, valid ones include: cheap budget, moderate budget, and expensive budget. Please try again.")

    preference = preference_list.strip().split(',')
    if(len(preference) > 1 ):
        raise ValueError("Attraction only allows one preference. Please try again")
    if '-' in preference[0]:
        preference_core = preference[0].strip().split('-')[0].lower()
    else:
        preference_core = preference[0].strip().split(' ')[0].lower()
    preferenceAccepted = ["family","history","activity","nature","food","shopping"]
    preferenceIsInRange = False
    if(preference_core in preferenceAccepted):
        preferenceIsInRange = True
    if not preferenceIsInRange:
        raise ValueError("Preference parameter invalid. Only family oriented / history oriented / activity oriented / nature oriented / food oriented / and shopping oriented are allowed. Please try again.")
    return True

def validate_restaurant_parameters_format(action_arg):
    pattern = r"(.*\s*.*),\s*(.*),\s*\[(.*)\]"
    match = re.match(pattern, action_arg)
    if not match:
        raise ValueError("Parameter format not match. Please try again. Valid Format: Budget, cuisine, preference list.")
    budget = match.group(1).lower()
    cuisine = match.group(2).lower()
    preference_list = match.group(3)

    budget_accepted = ['cheap budget', 'moderate budget','expensive budget']
    budgetInRange = False
    #print(budget)
    if budget in budget_accepted:
        budgetInRange = True
    if not budgetInRange:
        raise ValueError("Wrong budget Input, valid ones include: cheap budget, moderate budget and expensive budget. Please try again.")

    cuisine_accepted = ["us","mexican","irish","french","italian","greek","indian","chinese","japanese","korean","vietnamese","thai","asian fusion","middle eastern"]
    #print(cuisine)
    cuisineInRange = False
    if cuisine in cuisine_accepted:
        cuisineInRange = True
    if not cuisineInRange:
        raise ValueError("Cuisine not valid. Accepted cuisine is: US / Mexican / Irish / French / Italian / Greek / Indian / Chinese / Japanese / Korean / Vietnamese / Thai / Asian Fusion and Middle Eastern. Please try again.")

    preference_list = [p.lower().strip() for p in preference_list.split(',')]
    preference_core = [p.strip().split(' ')[-1] for p in preference_list]
    #print(preference_core)

    preferenceInRange = True
    preferenceAccepted = ['',"flavor","freshness","service","environment","value"]
    for p in preference_core:
        if p not in preferenceAccepted:
            preferenceInRange = False

    if not preferenceInRange:
        raise ValueError("Wrong preference Input. Accepted inputs are: good flavor / good freshness / good healthy/ good service / good environment / good value. Please try again.")    
    return True




if __name__ == "__main__":
    tools_list = ["attractions","accommodations","restaurants","nearby","planner"]
    modelList = ['gpt4o','mistral','llama318b']

    model = modelList[2]
    model_map = {'gpt4o': 'gpt-4o-2024-11-20', 'mistral':'mistral-large-2411','llama318b':'meta-llama/Llama-3.1-8B'}

    agent = ReactAgent(working_model = 'gpt4o', tools=tools_list, react_llm_name = model_map[model], planner_llm_name = model_map[model])
    toolUsePlans = []
    toolUseScratchpads = []
    toolUseLogs = []
    with open (f'Prompts/humanQuerys.jsonl', 'r') as file:
        humanquerys = [json.loads(line.strip()) for line in file]
    for i in range (1):
        query = humanquerys[i]['query']

        planner_results, scratchpad, action_log  = agent.run(query)

        toolUsePlans=[{"index": i+1, "plan": planner_results}]
        toolUseScratchpads=[{"index": i+1, "scratchpad": scratchpad}]
        toolUseLogs=[{"index": i+1, "log": action_log}]
        
        if(i%20 == 0):
            print(f'done with plan: {i}')

        with open (f'Output/{model}/plans/toolUsePlans.jsonl', 'a') as file:
            for plan in toolUsePlans:
                json.dump(plan, file)
                file.write('\n')
        with open (f'Output/{model}/plans/toolUseScratchpads.jsonl', 'a') as file:
            for scratchpad in toolUseScratchpads:
                json.dump(scratchpad, file)
                file.write('\n')
        with open (f'Output/{model}/plans/toolUseLogs.jsonl', 'a') as file:
            for log in toolUseLogs:
                json.dump(log, file)
                file.write('\n')
    """            
    with open (f'Output/{model}/plans/toolUsePlans.jsonl', 'a') as file:
        for plan in toolUsePlans:
            json.dump(plan, file)
            file.write('\n')
    with open (f'Output/{model}/plans/toolUseScratchpads.jsonl', 'a') as file:
        for scratchpad in toolUseScratchpads:
            json.dump(scratchpad, file)
            file.write('\n')
    with open (f'Output/{model}/plans/toolUseLogs.jsonl', 'a') as file:
        for log in toolUseLogs:
            json.dump(log, file)
            file.write('\n')"""

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
/home/jqf8qm/miniforge3/envs/torchgpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jqf8qm/miniforge3/envs/torchgpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Thought 1: I need to find the attractions that matches the preference.


...

Thought 4: I need to find the number of business clusters given the information that I've collected.
Action 4: BusinessClusterSearch[]


/home/jqf8qm/miniforge3/envs/torchgpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jqf8qm/miniforge3/envs/torchgpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Thought 5: I need to craft a detailed plan based on the information that I've collected.


/home/jqf8qm/miniforge3/envs/torchgpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jqf8qm/miniforge3/envs/torchgpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Action 5: Planner[Give me a 2-day trip plan in Philadelphia]
we begin to make a plan
the prompt is:  

You are a proficient travel planner. Based on the given information and query, you will generate a travel plan like the following example. Ensure that all recommendations and their addresses are organized in chronological order for each day. Give exactly 4 attraction recommendations for each day. Be considerate, concise and well-structured. Please also optimize the routes for the trip. For each day, find attractions that are close to each other for the recommendations. 

----- Example Starts -----

Query: I am planning a 2-day trip with an expensive budget. I would like to visit some history-oriented attractions. Please recommend Japanese restaurants with a good environment. For accommodation, I am looking for a hotel with good location, good quality, and good service.

Travel Plan: 
Day X:
- Accommodation: 
  - Name: XXXX
    Address: XXXX, XXXX

- Breakfast: 
  - Name: XXXX 
    Add

In [1]:
import torch
from transformers import pipeline

prompt="""You are a proficient travel planner. Based on the given information and query, you will generate a travel plan like the following example. Ensure that all recommendations and their addresses are organized in chronological order for each day. Give exactly 4 attraction recommendations for each day. Be considerate, concise and well-structured. Please also optimize the routes for the trip. For each day, find attractions that are close to each other for the recommendations. 

----- Example Starts -----

Query: I am planning a 2-day trip with an expensive budget. I would like to visit some history-oriented attractions. Please recommend Japanese restaurants with a good environment. For accommodation, I am looking for a hotel with good location, good quality, and good service.

Travel Plan: 
Day X:
- Accommodation: 
  - Name: XXXX
    Address: XXXX, XXXX

- Breakfast: 
  - Name: XXXX 
    Address: XXXX, XXXX

- Morning Attraction: 
  - Name: XXXX 
    Address: XXXX, XXXX 

- Lunch: 
  - Name: XXXX 
    Address: XXXX, XXXX 

- Afternoon Attraction: 
  - Name: XXXX 
    Address: XXXX, XXXX 
  - Name: XXXX
    Address: XXXX, XXXX

- Dinner: 
  - Name: XXXX 
    Address: XXXX, XXXX 

- Night Attraction: 
  - Name: XXXX

----- Example Ends -----

Given Information: [{'index': 0, 'Description': 'Attraction Choice', 'Content': "                                                            name                              address  latitude  longitude\n                                             76 Carriage Company                  1350 Schuylkill Ave 39.937574 -75.203767\n                                     Academy of Natural Sciences          1900 Benjamin Franklin Pkwy 39.957019 -75.171260\n                                          Auto Show Philadelphia                         1101 Arch St 39.954071 -75.159323\n                                                Awbury Arboretum                          1 Awbury Rd 40.050556 -75.168113\n                                        Benjamin Franklin Bridge                               US 676 39.952945 -75.134361\n                                        Benjamin Franklin Museum                      317 Chestnut St 39.949484 -75.146738\n                                              Birchtree Catering                        4500 Worth St 40.008612 -75.081733\n                                                   Bloktoberfest                    17 Washington Ave 39.943856 -75.167177\n                                                   Bow Tie Tours                    6TH And Market St 39.950616 -75.150347\n                                                Bury The Hatchet            1719 Washington Ave, Fl 2 39.938830 -75.172320\n                                              Cherry Street Pier                  121 N Columbus Blvd 39.952804 -75.139738\n                                          Chew Philly Food Tours                         4212 Main St 40.023867 -75.220830\n                                                       Chinatown                        North 12th St 39.976804 -75.154460\n                                                      Cira Green                         80 S 30th St 39.952673 -75.183339\n                                            City of Philadelphia                            1 Penn Sq 39.952394 -75.163599\n                                          David Thomas Trailways                     14005 McNulty Rd 40.106409 -74.973937\n                                                   Dilworth Park                          1 S 15th St 39.952505 -75.165131\n                        Eastern State Penitentiary Historic Site                   2027 Fairmount Ave 39.968368 -75.172655\n                                           Fairmount Water Works                    640 Waterworks Dr 39.966035 -75.183492\n                                            Fante's Kitchen Shop                        1006 S 9th St 39.937373 -75.158346\n                                                        FDR Park                    1500 Pattison Ave 39.905550 -75.175582\n                                     Fishtown Rivercity Festival                  1199 N Delaware Ave 39.966444 -75.130128\n                                                    Fort Mifflin                   6400 Hog Island Rd 39.875325 -75.213082\n                                              Founding Footsteps                          302 Arch St 39.952035 -75.145434\n                                       Franklin Square Mini Golf                         200 N 6th St 39.955848 -75.150486\n                                                 Franklin Square                         200 N 6th St 39.955700 -75.150137\n                                                Greensgrow Farms                 2501 E Cumberland St 39.978170 -75.120856\n                                           Harry Potter Festival                    16 E Highland Ave 40.070696 -75.210152\n                                      Historic Philadelphia, Inc               600 Walnut St, Ste 130 39.948321 -75.151952\n                                        Independence After Hours                      600 Chestnut St 39.949101 -75.149177\n                                     Independence Seaport Museum                  211 S Columbus Blvd 39.945918 -75.140474\n                                   Lincoln Financial Field Tours      One Lincoln Financial Field Way 39.900998 -75.167496\n                                          Manayunk Arts Festival                         4312 Main St 40.024630 -75.222518\n                                   Midtown Village Fall Festival                       1300 Walnut St 39.948887 -75.162323\n                                                Morris Arboretum               100 E Northwestern Ave 40.089742 -75.222909\n                               Museum of the American Revolution                         101 S 3rd St 39.948212 -75.145552\n                                    National Constitution Center                          525 Arch St 39.953893 -75.149068\n                                         National Liberty Museum                      321 Chestnut St 39.948830 -75.146832\n                      National Museum of American Jewish History            101 S Independence Mall E 39.950243 -75.148561\n                                     Neon Museum of Philadelphia           1800 N American St, Unit E 39.978546 -75.139660\n                                                NEST Center City                       1301 Locust St 39.948044 -75.161647\n                                               Old City District                        231 Market St 39.950286 -75.144751\n                                                   Palace Travel             123 S Broad St, Ste 1040 39.949823 -75.163825\n                                     Parks On Tap-Azalea Gardens                    636 Waterworks Dr 39.968375 -75.183846\n                                            Patriot Harbor Lines                  211 S Columbus Blvd 39.945796 -75.140583\n                                                     Penn Museum                        3260 South St 39.949477 -75.191784\n                                                  Penn's Landing                 Blue Cross River Rin 39.947948 -75.139738\n                               Pennsylvania Academy of Fine Arts                   118-128 N Broad St 39.955637 -75.163490\n                   Philadelphia Insectarium & Butterfly Pavilion                   8046 Frankford Ave 40.042510 -75.026367\nPhiladelphia International Festival of the Arts PIFA Street Fair          300 S Broad St Philadelphia 39.946622 -75.164951\n                                      Philadelphia Museum of Art          2600 Benjamin Franklin Pkwy 39.965573 -75.180969\n                  Philadelphia Segway Tours By Wheel Fun Rentals              1 N Independence Mall W 39.951283 -75.150002\n                 Philadelphia Sightseeing Tours & Transportation            111 S Independence Mall E 39.949765 -75.148773\n                                   Philadelphia Urban Adventures                          229 Arch St 39.952091 -75.140140\n                                                Philadelphia Zoo                    3400 W Girard Ave 39.971992 -75.196271\n                                                Philly Bike Tour                   2015 Fairmount Ave 39.967468 -75.170830\n                                           Philly Official Guide                          229 Arch St 39.936754 -75.164794\n                                                 Philly Tour Hub                   7 N. Columbus Blvd 39.951459 -75.140206\n                                                     Pizza Brain                   2313 Frankford Ave 39.980675 -75.128097\n                                             Please Touch Museum             4231 Ave Of The Republic 39.979483 -75.209010\n                                                     Powel House                         244 S 3rd St 39.945759 -75.146927\n                                         Reading Terminal Market                         51 N 12th St 39.953341 -75.158855\n                                                 Restaurant Week                      660 Chestnut St 39.952335 -75.163789\n                                     Rittenhouse Square District                 210 W Rittenhouse Sq 39.949400 -75.172100\n                                         Rittenhouse Square Park                       1800 Walnut St 39.949479 -75.171888\n                                          RiverLink Ferry System          Coulmbus Blvd And Walnut St 39.944086 -75.132151\n                                                  Rizzo Ice Rink                      1001 S Front St 39.933191 -75.145625\n                                                  Seger Dog Park                       1001 Rodman St 39.943364 -75.159817\n                                             Shane Confectionery                        110 Market St 39.949547 -75.142698\n                                 Shofuso Japanese House & Garden    Horticultural Dr And Lansdowne Dr 39.981419 -75.213578\n                            Simeone Foundation Automotive Museum                  6825-31 Norwitch Dr 39.910601 -75.226409\n                                              Sister Cities Park 18th St & the Benjamin Franklin Pkwy 39.957945 -75.169675\n                                 South Street Headhouse District                   2nd And Lombard St 39.942175 -75.145318\n                                          Spirit of Philadelphia                  401 S Columbus Blvd 39.941598 -75.140577\n                                      Spirits of '76 Ghost Tours                      325 Chestnut St 39.949088 -75.147154\n                                       Spruce Street Harbor Park                  121 N Columbus Blvd 39.944542 -75.141475\n                                    StrEATS of Philly Food Tours           1500 Market St, Ste W-3110 39.952317 -75.166486\n                                                          Suraya                   1528 Frankford Ave 39.973687 -75.133956\n                                                    Tattooed Mom                         530 South St 39.941680 -75.151916\n                     The African American Museum in Philadelphia                          701 Arch St 39.953261 -75.151756\n                          The American Swedish Historical Museum                    1900 Pattison Ave 39.905951 -75.181053\n                                            The Betsy Ross House                          239 Arch St 39.952280 -75.144610\n                                             The Big Bus Company                        600 Market St 39.950512 -75.150768\n                                 The Constitutional Walking Tour                          525 Arch St 39.954025 -75.148883\n                           The Franklin Institute Science Museum                        222 N 20th St 39.958211 -75.173137\n                                                   The Navy Yard                      4747 S Broad St 39.893152 -75.174557\n                                                        The Oval                          Eakins Oval 39.963766 -75.178530\n                                                      The Piazza                        1001 N 2nd St 39.966561 -75.139990\n                                    University Family Fun Center                       4006 Spruce St 39.951538 -75.203486\n                                           Walnut Lane Golf Club                        800 Walnut Ln 40.030227 -75.202825\n                                       Wonderspaces Philadelphia                         27 N 11th St 39.952297 -75.157672\n                                             Woodmere Art Museum                  9201 Germantown Ave 40.083186 -75.219418"}, {'index': 1, 'Description': 'Restaurant Choice', 'Content': "                            name                address  latitude  longitude            good_for_meal\n               Bing Bing Dim Sum    1648 E Passyunk Ave 39.928990 -75.164742                   dinner\n                        Buddakan        325 Chestnut St 39.948919 -75.147167                   dinner\n                         Dan Dan          126 S 16th St 39.950247 -75.167598            lunch, dinner\n                   Dim Sum House 3939 Chestnut St, Fl 2 39.955771 -75.201654            lunch, dinner\n                E Mei Restaurant            915 Arch St 39.953682 -75.155427            lunch, dinner\n                     Han Dynasty         3711 Market St 39.956711 -75.197103            lunch, dinner\n                     Han Dynasty        123 Chestnut St 39.948526 -75.143728                   dinner\nNan Zhou Hand Drawn Noodle House           1022 Race St 39.955401 -75.156901            lunch, dinner\n            Nom Wah Philadelphia          218 N 13th St 39.956593 -75.160772                    lunch\n         Reading Terminal Market           51 N 12th St 39.953341 -75.158855 lunch, brunch, breakfast\n                   Su Xing House         1508 Sansom St 39.950215 -75.166346            lunch, dinner\n                   Tom's Dim Sum           59 N 11th St 39.953289 -75.157854            lunch, dinner"}, {'index': 2, 'Description': 'Accommodation Choice', 'Content': "                                                     name              address  latitude  longitude\nCourtyard by Marriott Philadelphia South at The Navy Yard    1001 Intrepid Ave 39.895455 -75.171726\n           Independence Park Hotel, Bw Premier Collection      235 Chestnut St 39.948669 -75.145468\n                       Kimpton Hotel Palomar Philadelphia        117 S 17th St 39.950829 -75.168629\n                                              Lokal Hotel         139 N 3rd St 39.953255 -75.144790\n                                        Penn's View Hotel        14 N Front St 39.950201 -75.142285\n                                    ROOST Apartment Hotel        111 S 15th St 39.950568 -75.165702\n                                             The Dwight D        256 S 16th St 39.947989 -75.168277\n                                  The Franklin Residences      834 Chestnut St 39.949383 -75.155131\n                          The Inn at Penn, a Hilton Hotel       3600 Sansom St 39.953783 -75.195782\n                                    The Rittenhouse Hotel 210 W Rittenhouse Sq 39.949913 -75.173645"}, {'index': 3, 'Description': 'Business Cluster Results', 'Content': '{\'Cluster_0\': [[\'Shane Confectionery\'], [\'The Constitutional Walking Tour\'], [\'Franklin Square\'], [\'Franklin Square Mini Golf\'], [\'Founding Footsteps\'], [\'The Big Bus Company\'], [\'The Betsy Ross House\'], [\'Museum of the American Revolution\'], [\'National Constitution Center\'], [\'National Liberty Museum\'], [\'National Museum of American Jewish History\'], [\'The African American Museum in Philadelphia\'], [\'Old City District\'], [\'Independence After Hours\'], [\'Independence Park Hotel, Bw Premier Collection\'], [\'Philadelphia Segway Tours By Wheel Fun Rentals\'], [\'Lokal Hotel\'], [\'Cherry Street Pier\'], [\'Philadelphia Sightseeing Tours & Transportation\'], [\'Bow Tie Tours\'], [\'Philadelphia Urban Adventures\'], ["Spirits of \'76 Ghost Tours"], [\'Benjamin Franklin Museum\'], [\'Philly Tour Hub\'], [\'Powel House\'], ["Penn\'s View Hotel"], [\'The Franklin Residences\'], [\'Tattooed Mom\'], [\'Historic Philadelphia, Inc\']], \'Cluster_1\': [[\'Morris Arboretum\'], [\'Harry Potter Festival\'], [\'Woodmere Art Museum\']], \'Cluster_10\': [[\'Awbury Arboretum\']], \'Cluster_11\': [[\'Birchtree Catering\']], \'Cluster_12\': [[\'Fort Mifflin\']], \'Cluster_13\': [[\'Independence Seaport Museum\'], [\'Rizzo Ice Rink\'], [\'RiverLink Ferry System\'], [\'South Street Headhouse District\'], [\'Spirit of Philadelphia\'], [\'Benjamin Franklin Bridge\'], ["Penn\'s Landing"], [\'Patriot Harbor Lines\'], [\'Spruce Street Harbor Park\']], \'Cluster_14\': [[\'Philly Official Guide\'], [\'Seger Dog Park\'], [\'Bury The Hatchet\'], [\'Bloktoberfest\'], ["Fante\'s Kitchen Shop"]], \'Cluster_15\': [[\'Greensgrow Farms\'], [\'Suraya\'], [\'Neon Museum of Philadelphia\'], [\'Chinatown\'], [\'Pizza Brain\']], \'Cluster_16\': [[\'The Franklin Institute Science Museum\'], [\'The Oval\'], [\'Fairmount Water Works\'], [\'Cira Green\'], [\'Philly Bike Tour\'], [\'Eastern State Penitentiary Historic Site\'], [\'Parks On Tap-Azalea Gardens\'], [\'Philadelphia Museum of Art\']], \'Cluster_17\': [[\'Please Touch Museum\'], [\'Shofuso Japanese House & Garden\']], \'Cluster_18\': [[\'The Inn at Penn, a Hilton Hotel\'], [\'University Family Fun Center\'], [\'Penn Museum\']], \'Cluster_19\': [[\'Simeone Foundation Automotive Museum\']], \'Cluster_2\': [[\'Philadelphia Insectarium & Butterfly Pavilion\']], \'Cluster_3\': [[\'76 Carriage Company\']], \'Cluster_4\': [[\'Rittenhouse Square District\'], [\'Academy of Natural Sciences\'], [\'The Dwight D\'], [\'ROOST Apartment Hotel\'], [\'Auto Show Philadelphia\'], [\'Kimpton Hotel Palomar Philadelphia\'], [\'City of Philadelphia\'], [\'Wonderspaces Philadelphia\'], [\'Dilworth Park\'], [\'Rittenhouse Square Park\'], [\'Midtown Village Fall Festival\'], [\'The Rittenhouse Hotel\'], [\'Sister Cities Park\'], [\'NEST Center City\'], [\'Restaurant Week\'], [\'Palace Travel\'], [\'Reading Terminal Market\'], [\'Pennsylvania Academy of Fine Arts\'], [\'StrEATS of Philly Food Tours\'], [\'Philadelphia International Festival of the Arts PIFA Street Fair\']], \'Cluster_5\': [[\'The Navy Yard\'], [\'Lincoln Financial Field Tours\'], [\'FDR Park\'], [\'The American Swedish Historical Museum\'], [\'Courtyard by Marriott Philadelphia South at The Navy Yard\']], \'Cluster_6\': [[\'David Thomas Trailways\']], \'Cluster_7\': [[\'Walnut Lane Golf Club\'], [\'Chew Philly Food Tours\'], [\'Manayunk Arts Festival\']], \'Cluster_8\': [[\'Philadelphia Zoo\']], \'Cluster_9\': [[\'The Piazza\'], [\'Fishtown Rivercity Festival\']]}'}]

Query: Give me a 2-day trip plan in Philadelphia

Travel Plan: """


In [2]:
llm = pipeline(
    "text-generation", model="meta-llama/Llama-3.1-8B", model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [10]:
request = llm(prompt, return_full_text = False)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [12]:
request

[{'generated_text': ''}]